In [68]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import time
import pandas as pd
import re

In [91]:
webtoon = pd.read_csv("/Users/kh/Documents/ML_project/웹툰 추천 시스템/data/webtoon.csv")
title_data = list(webtoon["title"])

In [357]:
driver = webdriver.Chrome()
driver.get("https://comic.naver.com/webtoon")
time.sleep(2)

In [18]:
def getDaysData(id_list, title_list, day_list, author_list, rating_list, desc_list, genre_list, finish_list):
    html = driver.page_source # 가져온 페이지의 source를 저장
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.select("#content > div:nth-child(1) > ul > li")
    for idx, item in enumerate(items):
        #id 추출
        text = item.select_one("a")["href"]
        id = re.search(r'titleId=(\d+)', text).group(1)
        
        # 제목 추출
        title = item.select_one("div > a > span > span").text.strip()
        # 저자 정보가 있는 여러 위치를 체크하여 첫 번째로 발견된 요소를 사용
        author_element = item.select_one("div > div.ContentAuthor__author_wrap--fV7Lo > a, div > a.ContentAuthor__author--CTAAP")
        author = author_element.text.strip() if author_element else "Author Not Found"
        
        # 평점 추출
        rating = item.select_one("div > div.rating_area > span > span").text.strip()
        
        wait = WebDriverWait(driver, 3)  # 10초 동안 대기
        element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"#content > div:nth-child(1) > ul > li:nth-child({str(idx+1)}) > div > a > span > span")))
        element.click()
        time.sleep(3)
        day = driver.find_element(By.CSS_SELECTOR, "#content > div.EpisodeListInfo__comic_info--yRAu0 > div > div.ContentMetaInfo__meta_info--GbTg4 > em").text.split("\n∙\n")[0]
        description = driver.find_element(By.CSS_SELECTOR, "#content > div.EpisodeListInfo__comic_info--yRAu0 > div > div.EpisodeListInfo__summary_wrap--ZWNW5 > p").text
        genre = driver.find_element(By.CSS_SELECTOR, "#content > div.EpisodeListInfo__comic_info--yRAu0 > div > div.EpisodeListInfo__summary_wrap--ZWNW5 > div > div > a:nth-child(1)").text[1:]
        driver.back()
        time.sleep(3)
        finish = "ongoing"

        id_list.append(id)
        title_list.append(title)
        author_list.append(author)
        rating_list.append(rating)
        day_list.append(day)
        desc_list.append(description)
        genre_list.append(genre)
        finish_list.append(finish)

        print(title)

    return id_list, title_list, day_list, author_list, rating_list, desc_list, genre_list, finish_list

In [ ]:
#content > div:nth-child(1) > ul > li:nth-child(2085)

In [102]:
def getFinData(id_list, title_list, day_list, author_list, rating_list, desc_list, genre_list, finish_list):
    html = driver.page_source # 가져온 페이지의 source를 저장
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.select("#content > div:nth-child(1) > ul > li")
    count = 0
    driver.implicitly_wait(10)
    wait = WebDriverWait(driver, 15) # 15초간 대기
    for idx, item in enumerate(items):
        #id 추출
        text = item.select_one("a")["href"]
        id = re.search(r'titleId=(\d+)', text).group(1)
        
        # 제목 추출
        title = item.select_one("div > a > span > span").text.strip()
        # 저자 정보가 있는 여러 위치를 체크하여 첫 번째로 발견된 요소를 사용
        author_element = item.select_one("div > div.ContentAuthor__author_wrap--fV7Lo > a, div > a.ContentAuthor__author--CTAAP")
        author = author_element.text.strip() if author_element else "Author Not Found"
        
        # 평점 추출
        rating = item.select_one("div > div.rating_area > span > span").text.strip()
        element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"#content > div:nth-child(1) > ul > li:nth-child({str(idx+1)}) > div > a > span > span")))
        element.click()
        time.sleep(1)
        description = driver.find_element(By.CSS_SELECTOR, "#content > div.EpisodeListInfo__comic_info--yRAu0 > div > div.EpisodeListInfo__summary_wrap--ZWNW5 > p").text
        genre = driver.find_element(By.CSS_SELECTOR, "#content > div.EpisodeListInfo__comic_info--yRAu0 > div > div.EpisodeListInfo__summary_wrap--ZWNW5 > div > div > a:nth-child(1)").text[1:]
        driver.back()
        finish = "Complited"
        day = "매일 연재"
        
        count += 1
        if count % 50 == 0:
            time.sleep(5)
        id_list.append(id)
        title_list.append(title)
        author_list.append(author)
        rating_list.append(rating)
        day_list.append(day)
        desc_list.append(description)
        genre_list.append(genre)
        finish_list.append(finish)

        print(title)
        print(count)

    return id_list, title_list, day_list, author_list, rating_list, desc_list, genre_list, finish_list

In [355]:
driver = webdriver.Chrome()
url = "https://comic.naver.com/webtoon"
driver.get(url)

In [19]:
# 요일별 웹툰 데이터 수집

id_list = []
title_list = []
day_list = []
author_list = []
rating_list = []
desc_list = []
genre_list = []
finish_list = []

for dayNum in range(2, 10):
    driver.find_element(By.CSS_SELECTOR, f"#wrap > header > div.SubNavigationBar__snb_wrap--A5gfM > nav > ul > li:nth-child({dayNum}) > a").click()
    time.sleep(3)
    getDaysData(id_list, title_list, day_list, author_list, rating_list, desc_list, genre_list, finish_list)
ongoing_webtoon = pd.DataFrame({
    "Id": id_list,
    "title": title_list,
    "genre": genre_list,
    "author": author_list,
    "desc": desc_list,
    "rating": rating_list,
    "day": day_list,
    "finish": finish_list
})

ongoing_webtoon.to_csv("/Users/kh/Documents/ML_project/웹툰 추천 시스템/data/ongoing_webtoon.csv", index = False)

뷰티풀 군바리
참교육
백수세끼
신의 탑
왕과의 야행
윈드브레이커
퀘스트지상주의
소녀의 세계
신화급 귀속 아이템을 손에 넣었다
버림받은 왕녀의 은밀한 침실
귀촌리
절대검감
팔이피플
환생천마
별난식당
수상한 다이어트 클럽
우아한 욕망
회귀한 공작가의 막내도련님은 암살자
어린이집 다니는 구나
만렙돌파
토마토가 돼라!
진주
북부 공작님을 유혹하겠습니다
마왕을 그만둔 이유
개같이 탈출
인섹터
제왕
히어로메이커
더블클릭
악당 가족이 독립을 반대한다
리턴 투 플레이어
막장 악녀
영업 천재가 되었다
칼가는 소녀
녹빛자정의 연인
좀비묵시록 82-08
미친 재능의 플레이어
여고생이 신인데 나만 괴롭힘
K학원 생존기
결혼생활 그림일기
루루라라 우리네 인생
달이 뜨지 않는 도시
아슈타르테
오빠집이 비어서
파운더
보스리턴
헬크래프트
꿈의 기업
시한부의 아이까지 뺏으려 합니다
함부로 친절하지 말라
찔레꽃 그늘 아래
사랑, 그거 어떻게 하는 건데
공녀님의 꽃밭에는 그들이 산다
악녀교실
일파만파
엘프
제국 제일의 상속녀가 되었습니다
학식의 꿈
내가 왜 킬러?!
영앤리치가 아니야!
두 번째 삶은 힐링라이프?
사이다걸
반려짐승
달로 만든 아이
백호랑
고백어택
집사, 주세요!
찌질하지만 로맨스는 하고 싶어
마이너스의 손
최후의 금빛아이
다육이는 잘 자란다
어쌔신 크리드 - 잊혀진 사원
프로페서
컨트롤X
날 먹는 건 금지양!
나의 보이소프렌드
역주행!
주작연애
오늘의 일기예보
티엔다비스 - 완벽한 구원을 위하여
처음을 줄게!
선빵필승!
버그이터
악취해결사
어느날 짝남에게 공작님이 빙의했다
트리거
2024 내일 뭐 입지?
요리사가 되고 싶은 천마님
내가 쓴 소설 속 주인공들이 나를 죽이려 드는데요?
좋아? 죽어!
김부장
마음의소리2
서울 자가에 대기업 다니는 김 부장 이야기
마루는 강쥐
내가 키운 S급들
괴력 난신
유부 감자
피부과 만렙남
사신소년
멸망 이후의 세계
한림체육관
천마는 평범하게 살 수 없다
이섭의 연애
집이 없어
하루만 네가 되고 싶어
욕망일기Deep
유사연애
신입사원 김철수
하북팽

In [27]:
ongoing_webtoon.to_csv("/Users/kh/Documents/ML_project/웹툰 추천 시스템/data/ongoing_webtoon.csv", index = False)

In [125]:
# 완결웹툰 데이터 수집
id_list = []
title_list = []
day_list = []
author_list = []
rating_list = []
desc_list = []
genre_list = []
finish_list = []

In [362]:
html = driver.page_source # 가져온 페이지의 source를 저장
soup = BeautifulSoup(html, 'html.parser')
items = soup.select("#content > div:nth-child(1) > ul > li")
count = 0
driver.implicitly_wait(30)
wait = WebDriverWait(driver, 30) # 15초간 대기

for idx, item in enumerate(items):
    #id 추출
    text = item.select_one("a")["href"]
    id = re.search(r'titleId=(\d+)', text).group(1)
    
    # 제목 추출
    title = item.select_one("div > a > span > span").text.strip()
    # 저자 정보가 있는 여러 위치를 체크하여 첫 번째로 발견된 요소를 사용
    author_element = item.select_one("div > div.ContentAuthor__author_wrap--fV7Lo > a, div > a.ContentAuthor__author--CTAAP")
    author = author_element.text.strip() if author_element else "Author Not Found"
    
    # 평점 추출
    rating = item.select_one("div > div.rating_area > span > span").text.strip()
    element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"#content > div:nth-child(1) > ul > li:nth-child({str(idx+2061)}) > div > a > span > span")))
    element.click()
    time.sleep(1)
    description = driver.find_element(By.CSS_SELECTOR, "#content > div.EpisodeListInfo__comic_info--yRAu0 > div > div.EpisodeListInfo__summary_wrap--ZWNW5 > p").text
    genre = driver.find_element(By.CSS_SELECTOR, "#content > div.EpisodeListInfo__comic_info--yRAu0 > div > div.EpisodeListInfo__summary_wrap--ZWNW5 > div > div > a:nth-child(1)").text[1:]
    driver.back()
    finish = "Complited"
    day = "매일 연재"
    
    count += 1
    if count % 50 == 0:
        time.sleep(5)
    
    id_list.append(id)
    title_list.append(title)
    author_list.append(author)
    rating_list.append(rating)
    day_list.append(day)
    desc_list.append(description)
    genre_list.append(genre)
    finish_list.append(finish)
    
    print(title, count)

내일은 럭키곰스타 2060
N의등대-thecalling 2061
3단합체김창남 2062
골방환상곡 2063
꿈의카페영다방 2064
반지의제왕 2065
와탕카2 2066
향수 2067
개편 축하 릴레이 카툰 2068
에피소드메이비 2069
고맙습니다 2070
아이러브올림픽게임즈 2071
취업의소리 2072
태왕광개토 2073
제로 2074
트라우마 2075
식스센스 2076
탐구생활2 2077
구석구석캠페인 2078
체스아일 2079
펫다이어리 2080
기다리다미쳐 2081
도자기 2082
AA 2083
사랑in 2084
까뱅 2085
와탕카 2086
타투 2087


In [363]:
complited_webtoon = pd.DataFrame({
    "Id": id_list,
    "title": title_list,
    "genre": genre_list,
    "author": author_list,
    "desc": desc_list,
    "rating": rating_list,
    "day": day_list,
    "finish": finish_list
})

complited_webtoon.to_csv("/Users/kh/Documents/ML_project/웹툰 추천 시스템/data/complited_webtoon.csv", index = False)

In [51]:
class scraper:
    def __init__(self, url, base_url):
        self.url = url
        self.base_url = base_url
        self.driver = webdriver.Chrome()
        self.soup = None
        self.data = []

    def fetchPage(self):
        self.driver.get(self.url)
        time.sleep(3)
        self.soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        
    def parseTAR(self): # T: title, A: author, R: rating
        if self.soup is not None:
            items = self.soup.select("#content > div:nth-child(1) > ul > li")
            for item in items:
                # 제목 추출
                title = item.select_one("div > a > span > span").text.strip()
                if title not in title_list:
                    # 저자 정보가 있는 여러 위치를 체크하여 첫 번째로 발견된 요소를 사용
                    author_element = item.select_one("div > div.ContentAuthor__author_wrap--fV7Lo > a, div > a.ContentAuthor__author--CTAAP")
                    author = author_element.text.strip() if author_element else "Author Not Found"
                    
                    # 평점 추출
                    rating = item.select_one("div > div.rating_area > span > span")

                    self.data.append({
                    'title': title,
                    'author': author,
                    'rating': rating})
        else:
            print("Soup is None. Call fetch_page() first")

    def parseGD(self): # G: genre, D: description
        self.driver.get(self.base_url + self.url)
        self.soup = BeautifulSoup(self.driver.page_source, 'html.parser')
                            

In [ ]:
#content > div:nth-child(1) > ul > li:nth-child(1) > div > a > span > span
#content > div:nth-child(1) > ul > li:nth-child(2) > div > a > span > span

#content > div:nth-child(1) > ul > li:nth-child(1) > div > div.ContentAuthor__author_wrap--fV7Lo > a
#content > div:nth-child(1) > ul > li:nth-child(2) > div > div.ContentAuthor__author_wrap--fV7Lo > a

#content > div:nth-child(1) > ul > li:nth-child(1) > div > div.rating_area > span > span
#content > div:nth-child(1) > ul > li:nth-child(2) > div > div.rating_area > span > span

#content > div:nth-child(1) > ul > li:nth-child(3) > div > a.ContentAuthor__author--CTAAP

#content > div:nth-child(1) > ul > li:nth-child(1) > div > a > span